In [ ]:
# coding: utf-8

from __future__ import division
import numpy as np
import pandas as pd
from sklearn import metrics
import lightgbm as lgb
import time
from multiprocessing import cpu_count
import warnings
from pandas import set_option
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('./../data/uid_train.txt', header=None, sep='\t')
train.columns = ['uid', 'label']
train_voice = pd.read_csv('./../data/voice_train.txt', header=None, sep='\t')
train_voice.columns = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'end_time', 'call_type', 'in_out']
train_sms = pd.read_csv('./../data/sms_train.txt', header=None, sep='\t')
train_sms.columns = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'in_out']
train_wa = pd.read_csv('./../data/wa_train.txt', header=None, sep='\t')
train_wa.columns = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'wa_type', 'date']

test_a = pd.DataFrame({'uid': ['u' + str(i) for i in range(5000, 7000)]})
test_a_voice = pd.read_csv('./../data/voice_test_a.txt', header=None, sep='\t')
test_a_voice.columns = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'end_time', 'call_type', 'in_out']
test_a_sms = pd.read_csv('./../data/sms_test_a.txt', header=None, sep='\t')
test_a_sms.columns = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'in_out']
test_a_wa = pd.read_csv('./../data/wa_test_a.txt', header=None, sep='\t')
test_a_wa.columns = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'wa_type', 'date']

test_b = pd.DataFrame({'uid': ['u' + str(i) for i in range(7000, 10000)]})
test_b_voice = pd.read_csv('./../data/voice_test_b.txt', header=None, sep='\t')
test_b_voice.columns = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'end_time', 'call_type', 'in_out']
test_b_sms = pd.read_csv('./../data/sms_test_b.txt', header=None, sep='\t')
test_b_sms.columns = ['uid', 'opp_num', 'opp_head', 'opp_len', 'start_time', 'in_out']
test_b_wa = pd.read_csv('./../data/wa_test_b.txt', header=None, sep='\t')
test_b_wa.columns = ['uid', 'wa_name', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow', 'wa_type', 'date']


df = pd.concat([train, test_a, test_b]).reset_index(drop=True) # label + uid,0~9999
df_voice = pd.concat([train_voice, test_a_voice, test_b_voice]).reset_index(drop=True)
df_sms = pd.concat([train_sms, test_a_sms, test_b_sms]).reset_index(drop=True)
df_wa = pd.concat([train_wa, test_a_wa, test_b_wa]).reset_index(drop=True)

In [ ]:
day,hour,minute = 24*60*60, 60*60, 60

# input_time1:结束时间，input_time2:开始时间
def cal_voice_time(input_time1, input_time2):
    in_str1 = str(input_time1)
    in_str2 = str(input_time2)
    day_time1 = 0
    day_time2 = 0
    hour_time = int(in_str1[-6:-4]) - int(in_str2[-6:-4])
    minute_time = int(in_str1[-4:-2]) - int(in_str2[-4:-2])
    second_time = int(in_str1[-2:]) - int(in_str2[-2:])
    
    if (len(str(input_time1)) == 8):
        day_time1 = int(in_str1[-8:-6])
    elif(len(str(input_time1)) == 7):
        day_time1 = int(in_str1[-7])

    if (len(str(input_time2)) == 8):
        day_time2 = int(in_str2[-8:-6])
    elif(len(str(input_time2)) == 7):
        day_time2 = int(in_str2[-7])

    day_time = day_time1 - day_time2

    total_voice_time = day_time*day + hour_time*hour + minute_time*minute + second_time
    return total_voice_time

def cal_date(input_time1):
    in_str1 = str(input_time1)
    day_time1 = 0
    if (len(str(input_time1)) == 8):
        day_time1 = int(in_str1[-8:-6])
    elif(len(str(input_time1)) == 7):
        day_time1 = int(in_str1[-7])
    
    return day_time1

In [ ]:
for i in range(0, df_voice.shape[0]):
    df_voice.at[i,'voice_time'] = cal_voice_time(df_voice.at[i,'end_time'], df_voice.at[i,'start_time'])
    
df_voice['date'] = 0
for i in range(0, df_voice.shape[0]):
    df_voice.at[i,'date'] = cal_date(df_voice.at[i,'start_time'])
    
df_voice['time_period'] = 0
for i in range(0, df_voice.shape[0]):
    df_voice.at[i,'time_period'] = int(str(df_voice.at[i,'start_time'])[-6:-4]) / 2 + 1
    
df_sms['date'] = 0
for i in range(0, df_sms.shape[0]):
    df_sms.at[i,'date'] = cal_date(df_sms.at[i,'start_time'])
    
df_sms['time_period'] = 0
for i in range(0, df_sms.shape[0]):
    df_sms.at[i,'time_period'] = int(str(df_sms.at[i,'start_time'])[-6:-4]) / 2 + 1

In [ ]:
# df_voice - feature_select

In [ ]:
voice_opp_num = df_voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()
voice_opp_head=df_voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

In [ ]:
voice_count_opp_len=df_voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_count_opp_len_').reset_index().fillna(0)
voice_sum_opp_len = df_voice.groupby(['uid','opp_len'])['voice_time'].sum().unstack().add_prefix('voice_sum_opp_len_').reset_index().fillna(0)
voice_mean_opp_len = df_voice.groupby(['uid','opp_len'])['voice_time'].mean().unstack().add_prefix('voice_mean_opp_len_').reset_index().fillna(0)
voice_median_opp_len = df_voice.groupby(['uid','opp_len'])['voice_time'].median().unstack().add_prefix('voice_median_opp_len_').reset_index().fillna(0)
voice_max_opp_len = df_voice.groupby(['uid','opp_len'])['voice_time'].max().unstack().add_prefix('voice_max_opp_len_').reset_index().fillna(0)
voice_min_opp_len = df_voice.groupby(['uid','opp_len'])['voice_time'].min().unstack().add_prefix('voice_min_opp_len_').reset_index().fillna(0)
voice_std_opp_len = df_voice.groupby(['uid','opp_len'])['voice_time'].std().unstack().add_prefix('voice_std_opp_len_').reset_index().fillna(0)

In [ ]:
voice_count_call_type=df_voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_count_call_type_').reset_index().fillna(0)
voice_sum_call_type = df_voice.groupby(['uid','call_type'])['voice_time'].sum().unstack().add_prefix('voice_sum_call_type_').reset_index().fillna(0)
voice_mean_call_type = df_voice.groupby(['uid','call_type'])['voice_time'].mean().unstack().add_prefix('voice_mean_call_type_').reset_index().fillna(0)
voice_median_call_type = df_voice.groupby(['uid','call_type'])['voice_time'].median().unstack().add_prefix('voice_median_call_type_').reset_index().fillna(0)
voice_max_call_type = df_voice.groupby(['uid','call_type'])['voice_time'].max().unstack().add_prefix('voice_max_call_type_').reset_index().fillna(0)
voice_min_call_type = df_voice.groupby(['uid','call_type'])['voice_time'].min().unstack().add_prefix('voice_min_call_type_').reset_index().fillna(0)
voice_std_call_type = df_voice.groupby(['uid','call_type'])['voice_time'].std().unstack().add_prefix('voice_std_call_type_').reset_index().fillna(0)

In [ ]:
voice_count_in_out=df_voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_count_in_out_').reset_index().fillna(0)
voice_sum_in_out = df_voice.groupby(['uid','in_out'])['voice_time'].sum().unstack().add_prefix('voice_sum_in_out_').reset_index().fillna(0)
voice_mean_in_out = df_voice.groupby(['uid','in_out'])['voice_time'].mean().unstack().add_prefix('voice_mean_in_out_').reset_index().fillna(0)
voice_median_in_out = df_voice.groupby(['uid','in_out'])['voice_time'].median().unstack().add_prefix('voice_median_in_out_').reset_index().fillna(0)
voice_max_in_out = df_voice.groupby(['uid','in_out'])['voice_time'].max().unstack().add_prefix('voice_max_in_out_').reset_index().fillna(0)
voice_min_in_out = df_voice.groupby(['uid','in_out'])['voice_time'].min().unstack().add_prefix('voice_min_in_out_').reset_index().fillna(0)
voice_std_in_out = df_voice.groupby(['uid','in_out'])['voice_time'].std().unstack().add_prefix('voice_std_in_out_').reset_index().fillna(0)

In [ ]:
voice_count_in_out['voice_count_out_minus_in'] = voice_count_in_out['voice_count_in_out_1'] - voice_count_in_out['voice_count_in_out_0']

In [ ]:
voice_count_date = df_voice.groupby(['uid','date'])['voice_time'].count().unstack().add_prefix('voice_count_date_').reset_index().fillna(0)
voice_sum_date = df_voice.groupby(['uid','date'])['voice_time'].sum().unstack().add_prefix('voice_sum_date_').reset_index().fillna(0)
voice_max_date = df_voice.groupby(['uid','date'])['voice_time'].max().unstack().add_prefix('voice_max_date_').reset_index().fillna(0)
voice_min_date = df_voice.groupby(['uid','date'])['voice_time'].min().unstack().add_prefix('voice_min_date_').reset_index().fillna(0)
voice_mean_date = df_voice.groupby(['uid','date'])['voice_time'].mean().unstack().add_prefix('voice_mean_date_').reset_index().fillna(0)
voice_median_date = df_voice.groupby(['uid','date'])['voice_time'].median().unstack().add_prefix('voice_median_date_').reset_index().fillna(0)
voice_std_date = df_voice.groupby(['uid','date'])['voice_time'].std().unstack().add_prefix('voice_std_date_').reset_index().fillna(0)

In [ ]:
voice_time = df_voice.groupby(['uid'])['voice_time'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_time_').reset_index()
voice_frequency=df_voice.groupby(['uid','time_period'])['uid'].count().unstack().add_prefix('voice_frequency_').reset_index().fillna(0)

In [ ]:
# df_voice : optional_feature_list
'''
fea_1 = [
voice_opp_num,voice_opp_head,
voice_count_opp_len,voice_sum_opp_len,voice_mean_opp_len,voice_median_opp_len,voice_max_opp_len,voice_min_opp_len,voice_std_opp_len,
voice_count_call_type,voice_sum_call_type,voice_mean_call_type,voice_median_call_type,voice_max_call_type,voice_min_call_type,
voice_std_call_type,
voice_count_in_out,voice_sum_in_out,voice_mean_in_out,voice_median_in_out,voice_max_in_out,voice_min_in_out,voice_std_in_out,
voice_count_date,voice_sum_date,voice_max_date,voice_min_date,voice_mean_date,voice_median_date,voice_std_date,
voice_time, voice_frequency]
'''

In [ ]:
# # df_sms - feature_select

In [ ]:
sms_opp_num = df_sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()
sms_opp_head=df_sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

In [ ]:
sms_count_opp_len=df_sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_count_opp_len_').reset_index().fillna(0)
sms_sum_opp_len = df_sms.groupby(['uid','opp_len'])['uid'].sum().unstack().add_prefix('sms_sum_opp_len_').reset_index().fillna(0)
sms_mean_opp_len = df_sms.groupby(['uid','opp_len'])['date'].mean().unstack().add_prefix('sms_mean_opp_len_').reset_index().fillna(0)
sms_median_opp_len = df_sms.groupby(['uid','opp_len'])['date'].median().unstack().add_prefix('sms_median_opp_len_').reset_index().fillna(0)
sms_max_opp_len = df_sms.groupby(['uid','opp_len'])['date'].max().unstack().add_prefix('sms_max_opp_len_').reset_index().fillna(0)
sms_min_opp_len = df_sms.groupby(['uid','opp_len'])['date'].min().unstack().add_prefix('sms_min_opp_len_').reset_index().fillna(0)
sms_std_opp_len = df_sms.groupby(['uid','opp_len'])['date'].std().unstack().add_prefix('sms_std_opp_len_').reset_index().fillna(0)

In [ ]:
sms_count_in_out=df_sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_count_in_out_').reset_index().fillna(0)
sms_sum_in_out = df_sms.groupby(['uid','in_out'])['date'].sum().unstack().add_prefix('sms_sum_in_out_').reset_index().fillna(0)
sms_mean_in_out = df_sms.groupby(['uid','in_out'])['date'].mean().unstack().add_prefix('sms_mean_in_out_').reset_index().fillna(0)
sms_median_in_out = df_sms.groupby(['uid','in_out'])['date'].median().unstack().add_prefix('sms_median_in_out_').reset_index().fillna(0)
sms_max_in_out = df_sms.groupby(['uid','in_out'])['date'].max().unstack().add_prefix('sms_max_in_out_').reset_index().fillna(0)
sms_min_in_out = df_sms.groupby(['uid','in_out'])['date'].min().unstack().add_prefix('sms_min_in_out_').reset_index().fillna(0)
sms_std_in_out = df_sms.groupby(['uid','in_out'])['date'].std().unstack().add_prefix('sms_std_in_out_').reset_index().fillna(0)

In [ ]:
sms_count_in_out['sms_count_out_minus_in'] = sms_count_in_out['sms_count_in_out_1'] - sms_count_in_out['sms_count_in_out_0']

In [ ]:
sms_count_date = df_sms.groupby(['uid','date'])['date'].count().unstack().add_prefix('sms_count_date_').reset_index().fillna(0)
sms_sum_date = df_sms.groupby(['uid','date'])['date'].sum().unstack().add_prefix('sms_sum_date_').reset_index().fillna(0)
sms_max_date = df_sms.groupby(['uid','date'])['date'].max().unstack().add_prefix('sms_max_date_').reset_index().fillna(0)
sms_min_date = df_sms.groupby(['uid','date'])['date'].min().unstack().add_prefix('sms_min_date_').reset_index().fillna(0)
sms_mean_date = df_sms.groupby(['uid','date'])['date'].mean().unstack().add_prefix('sms_mean_date_').reset_index().fillna(0)
sms_median_date = df_sms.groupby(['uid','date'])['date'].median().unstack().add_prefix('sms_median_date_').reset_index().fillna(0)
sms_std_date = df_sms.groupby(['uid','date'])['date'].std().unstack().add_prefix('sms_std_date_').reset_index().fillna(0)

In [ ]:
sms_frequency = df_sms.groupby(['uid','time_period'])['uid'].count().unstack().add_prefix('sms_frequency_').reset_index().fillna(0)

In [ ]:
# df_sms : optional_feature_list
'''
fea_2 = [
sms_opp_num, sms_opp_head,
sms_count_opp_len,sms_sum_opp_len,sms_mean_opp_len,sms_median_opp_len,sms_max_opp_len,sms_min_opp_len,sms_std_opp_len,
sms_count_in_out,sms_sum_in_out,sms_mean_in_out,sms_median_in_out,sms_max_in_out,sms_min_in_out,sms_std_in_out,
sms_count_date,sms_sum_date,sms_mean_date,sms_median_date,sms_max_date,sms_min_date,sms_std_date,
sms_frequency
]
'''

In [ ]:
# df_wa - feature_select

In [ ]:
wa_name = df_wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()
visit_cnt = df_wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()
visit_dura = df_wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()
up_flow = df_wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()
down_flow = df_wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()
wa_count = df_wa.groupby(['uid','date'])['visit_cnt'].sum().unstack().add_prefix('wa_count_').reset_index().fillna(0)

In [ ]:
wa_sum_visit_cnt = df_wa.groupby(['uid','date'])['visit_cnt'].sum().unstack().add_prefix('wa_sum_visit_cnt_').reset_index().fillna(0)
wa_mean_visit_cnt = df_wa.groupby(['uid','date'])['visit_cnt'].mean().unstack().add_prefix('wa_mean_visit_cnt_').reset_index().fillna(0)
wa_median_visit_cnt = df_wa.groupby(['uid','date'])['visit_cnt'].median().unstack().add_prefix('wa_median_visit_cnt_').reset_index().fillna(0)
wa_max_visit_cnt = df_wa.groupby(['uid','date'])['visit_cnt'].max().unstack().add_prefix('wa_max_visit_cnt_').reset_index().fillna(0)
wa_min_visit_cnt = df_wa.groupby(['uid','date'])['visit_cnt'].min().unstack().add_prefix('wa_min_visit_cnt_').reset_index().fillna(0)
wa_std_visit_cnt = df_wa.groupby(['uid','date'])['visit_cnt'].std().unstack().add_prefix('wa_std_visit_cnt_').reset_index().fillna(0)

In [ ]:
wa_sum_visit_dura = df_wa.groupby(['uid','date'])['visit_dura'].sum().unstack().add_prefix('wa_sum_visit_dura_').reset_index().fillna(0)
wa_mean_visit_dura = df_wa.groupby(['uid','date'])['visit_dura'].mean().unstack().add_prefix('wa_mean_visit_dura_').reset_index().fillna(0)
wa_median_visit_dura = df_wa.groupby(['uid','date'])['visit_dura'].median().unstack().add_prefix('wa_median_visit_dura_').reset_index().fillna(0)
wa_max_visit_dura = df_wa.groupby(['uid','date'])['visit_dura'].max().unstack().add_prefix('wa_max_visit_dura_').reset_index().fillna(0)
wa_min_visit_dura = df_wa.groupby(['uid','date'])['visit_dura'].min().unstack().add_prefix('wa_min_visit_dura_').reset_index().fillna(0)
wa_std_visit_dura = df_wa.groupby(['uid','date'])['visit_dura'].std().unstack().add_prefix('wa_std_visit_dura_').reset_index().fillna(0)

In [ ]:
wa_sum_up_flow = df_wa.groupby(['uid','date'])['up_flow'].sum().unstack().add_prefix('wa_sum_up_flow_').reset_index().fillna(0)
wa_mean_up_flow = df_wa.groupby(['uid','date'])['up_flow'].mean().unstack().add_prefix('wa_mean_up_flow_').reset_index().fillna(0)
wa_median_up_flow = df_wa.groupby(['uid','date'])['up_flow'].median().unstack().add_prefix('wa_median_up_flow_').reset_index().fillna(0)
wa_max_up_flow = df_wa.groupby(['uid','date'])['up_flow'].max().unstack().add_prefix('wa_max_up_flow_').reset_index().fillna(0)
wa_min_up_flow = df_wa.groupby(['uid','date'])['up_flow'].min().unstack().add_prefix('wa_min_up_flow_').reset_index().fillna(0)
wa_std_up_flow = df_wa.groupby(['uid','date'])['up_flow'].std().unstack().add_prefix('wa_std_up_flow_').reset_index().fillna(0)

In [ ]:
wa_sum_down_flow = df_wa.groupby(['uid','date'])['down_flow'].sum().unstack().add_prefix('wa_sum_down_flow_').reset_index().fillna(0)
wa_mean_down_flow = df_wa.groupby(['uid','date'])['down_flow'].mean().unstack().add_prefix('wa_mean_down_flow_').reset_index().fillna(0)
wa_median_down_flow = df_wa.groupby(['uid','date'])['down_flow'].median().unstack().add_prefix('wa_median_down_flow_').reset_index().fillna(0)
wa_max_down_flow = df_wa.groupby(['uid','date'])['down_flow'].max().unstack().add_prefix('wa_max_down_flow_').reset_index().fillna(0)
wa_min_down_flow = df_wa.groupby(['uid','date'])['down_flow'].min().unstack().add_prefix('wa_min_down_flow_').reset_index().fillna(0)
wa_std_down_flow = df_wa.groupby(['uid','date'])['down_flow'].std().unstack().add_prefix('wa_std_down_flow_').reset_index().fillna(0)

In [ ]:
# df_wa : optional_feature_list
'''
fea_3 = [
wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_count,
wa_sum_visit_cnt,wa_mean_visit_cnt,wa_median_visit_cnt,wa_max_visit_cnt,wa_min_visit_cnt,wa_std_visit_cnt,
wa_sum_visit_dura,wa_mean_visit_dura,wa_median_visit_dura,wa_max_visit_dura,wa_min_visit_dura,wa_std_visit_dura,
wa_sum_up_flow,wa_mean_up_flow,wa_median_up_flow,wa_max_up_flow,wa_min_up_flow,wa_std_up_flow,
wa_sum_down_flow,wa_mean_down_flow,wa_median_down_flow,wa_max_down_flow,wa_min_down_flow,wa_std_down_flow
] 
'''

In [ ]:
# 0.779 - cv
feature = [
voice_opp_num,voice_opp_head,
voice_count_opp_len,#voice_sum_opp_len,voice_mean_opp_len,voice_max_opp_len,voice_min_opp_len,voice_std_opp_len,
voice_count_call_type,#voice_sum_call_type,voice_mean_call_type,voice_max_call_type,voice_min_call_type,voice_std_call_type,
voice_count_in_out,#voice_sum_in_out,voice_mean_in_out,voice_max_in_out,voice_min_in_out,voice_std_in_out,
voice_count_date,voice_sum_date,voice_max_date,voice_min_date,voice_mean_date,voice_std_date,
voice_time, voice_frequency,
    
sms_opp_num, sms_opp_head,
sms_count_opp_len,#sms_mean_opp_len,sms_median_opp_len,sms_max_opp_len,sms_min_opp_len,sms_std_opp_len,
sms_count_in_out,#sms_sum_in_out,sms_mean_in_out,sms_median_in_out,sms_max_in_out,sms_min_in_out,sms_std_in_out,
sms_count_date,#sms_sum_date,sms_mean_date,sms_median_date,sms_max_date,sms_min_date,sms_std_date,
sms_frequency,

wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_count,
wa_sum_visit_cnt,#wa_mean_visit_cnt,wa_median_visit_cnt,wa_max_visit_cnt,wa_min_visit_cnt,wa_std_visit_cnt,
wa_sum_visit_dura,#wa_mean_visit_dura,wa_median_visit_dura,wa_max_visit_dura,wa_min_visit_dura,wa_std_visit_dura,
wa_sum_up_flow,#wa_mean_up_flow,wa_median_up_flow,wa_max_up_flow,wa_min_up_flow,wa_std_up_flow,
wa_sum_down_flow,#wa_mean_down_flow,wa_median_down_flow,wa_max_down_flow,wa_min_down_flow,wa_std_down_flow
]

In [ ]:
for feat in feature:
    df = pd.merge(df,feat,how='left',on='uid')

In [ ]:
train_x = df.loc[:4998,:]
test_a_x = df.loc[4999:6998, :]
test_b_x = df.loc[6999:, :]

In [ ]:
dtrain = lgb.Dataset(train_x.drop(['uid','label'],axis=1),label=train_x.label)
dtest = lgb.Dataset(test_b_x.drop(['uid','label'],axis=1))

In [ ]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 12,
    'num_leaves': 64,
    'learning_rate': 0.08,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbosity':-1,
#    'gpu_device_id':2,
#    'device':'gpu'
#    'lambda_l1': 0.001,
#    'skip_drop': 0.95,
#    'max_drop' : 10
    #'lambda_l2': 0.005
    #'num_threads': 18
}    

In [ ]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)  
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    
    pre.preds=pre.preds.map(lambda x: 1 if x>=0.5 else 0)
    f1 = metrics.f1_score(pre.label,pre.preds)
    
    res = 0.6*auc +0.4*f1
    return 'res',res,True  

In [ ]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=10,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

In [ ]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=80,valid_sets=[dtrain])

In [ ]:
pred=model.predict(test_b_x.drop(['uid','label'],axis=1))

In [ ]:
res = pd.DataFrame({'uid':test_b_x.uid,'label':pred})
res=res.sort_values(by='label',ascending=False)
res.to_csv('./../data/tmp.csv', index=False)
res_tmp = pd.read_csv('./../data/tmp.csv')
for i in range(0, res_tmp.shape[0]):
    res_tmp.at[i,'label'] = 1 if(i<=540) else 0
res_tmp.label = res_tmp.label.map(lambda x: int(x))

In [ ]:
res_tmp.to_csv('./../sub/sub_lgb.csv',index=False,header=False,sep=',',columns=['uid','label'])